In [1]:
import numpy as np
import imageio
import cv2 
import glob
import matplotlib.pyplot as plt

## Create Images


In [125]:
circle_coordinates = np.arange(10,90,15)
out_images = []
counter  = 0
render_list = []
for x in circle_coordinates:
    image = np.zeros([100,100,3])
    image = cv2.circle(
	image, 
	(x, 45), 
	5, 
	(255, 255, 255), 
	5 
    )
    image = image.astype(np.uint8)
    plt.imsave("Images/" + str(counter) + ".png",image)
    render_list.append(image)
    counter += 1


In [126]:
imageio.mimsave('Renders/circles.gif', render_list, loop=65535)

## Import Images


In [127]:
images = sorted(glob.glob("Images/*.png"))
image_list = []
for image in images:
    temp = plt.imread(image,"png")
    image_list.append(temp.astype(float))

## Delay Image

In [123]:
render_list = []
for i in range(2,len(image_list) -2):
    previous_image = image_list[i -2]
    current_image = image_list[i]
    out_image = (current_image + previous_image)
    out_image *= 255
    out_image = out_image.astype(np.uint8)
    render_list.append(out_image)
    




In [124]:
imageio.mimsave('Renders/delay_image.gif', render_list, loop=65535)

## Image Comb Filter

In [130]:
size = [100,100]

images_copy = image_list.copy()

sr = 100 * 100
sr = sr // 2
hz = 100
dt_max = int((1 / 10) * sr)
feedback = -0.99
flat_size = int(images_copy[0].flatten().shape[0]) + dt_max
delay_buffer  = np.zeros(flat_size)
out_image =  np.zeros(100 * 100 * 4)
fb = np.zeros_like(out_image)
delayread = 0
num_iters  = (7 * 4) * 4
render_list = []
for iters in range(num_iters):
    for images in images_copy:
        image_flat = images.flatten()
        delay_buffer[dt_max:] = image_flat[:]
        dt = int(np.random.uniform(dt_max, sr // 2 - 1))
        for i in range(40000):
            delay_buffer[(delayread + dt) %flat_size] = image_flat[i]
            delayread += 1
        out_image[0: 100 * 100 * 4] =  0.33 * (image_flat + delay_buffer[0 :100 * 100 * 4] * 0.99 - fb * -0.99)
        fb = out_image * 0.99
        display = np.reshape(out_image,[100,100,4])
        display *= 255 
        render_list.append(display.astype(np.uint8))


In [131]:
imageio.mimsave('Renders/image_comb_filter.gif', render_list, loop=65535)